In [ ]:
%matplotlib notebook
import textwrap
import datetime

import requests
import overpy
import geopy.distance
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
BUFFER_SIZE = 150  # Max distance between schools and GCUM in m

In [ ]:
# Getting GCUM data
resp = requests.get('https://vigilo.jesuisundesdeux.org/get_issues.php?c=2')
gcum = resp.json()

In [ ]:
# Getting schools
api = overpy.Overpass()
bbox = '43.517, 3.777, 43.685, 4.035'
query = """(
  node["school:FR"="maternelle"]({bbox});
  way["school:FR"="maternelle"]({bbox});
  node["school:FR"="élémentaire"]({bbox});
  way["school:FR"="élémentaire"]({bbox});
  node["school:FR"="primaire"]({bbox});
  way["school:FR"="primaire"]({bbox});
  );
  out center;""".format(bbox=bbox)

result = api.query(query)

schools = [{'name': x.tags['name'], 'lat': x.lat, 'lon': x.lon} for x in result.nodes] + \
          [{'name': x.tags['name'], 'lat': x.center_lat, 'lon': x.center_lon} for x in result.ways]

In [ ]:
# Filtering gcum on day and time
week_gcum = [x for x in gcum
             if datetime.datetime.fromtimestamp(int(x['time'])).weekday() < 4
             and (datetime.time(hour=7, minute=30) < datetime.datetime.fromtimestamp(int(x['time'])).time() < datetime.time(hour=9)
                  or datetime.time(hour=16) < datetime.datetime.fromtimestamp(int(x['time'])).time() < datetime.time(hour=17, minute=30))]

In [ ]:
# Counting GCUM within X meters of each school
for school in schools:
    school['gcum'] = [x for x in week_gcum if geopy.distance.great_circle((x['coordinates_lat'], x['coordinates_lon']),
                                                                     (school['lat'],
                                                                      school['lon'])).meters < BUFFER_SIZE]
    school['nb_gcum'] = len(school['gcum'])

schools.sort(key=lambda x: len(x['gcum']), reverse=True)

print('\n'.join(['{} - {}'.format(x['nb_gcum'], x['name']) for x in schools if x['nb_gcum'] != 0]))

In [ ]:
sns.set_color_codes("pastel")
sns.barplot(x=[x['nb_gcum'] for x in schools[:10]],
            y=['\n'.join(textwrap.wrap(x['name'], 30)) for x in schools[:10]],
            color="b")

plt.title("Classement des écoles de Montpellier\npar nombre de GCUM à proximité\n(moins de {} m)".format(BUFFER_SIZE))

plt.tight_layout()
plt.show()